In [1]:
import descarteslabs as dl
import matplotlib.pyplot as plt
import geopandas as gpd

from math import cos
from collections import defaultdict

#### Load in the Iowa active sites dataset

In [2]:
active_sites_df = gpd.read_file('Iowa CAFOs/active_sites.geojson')
active_sites_df

,facName,LocAddress,CityName,State,locZip,OperatType,Swine,CattleDairy,CattleBeef,Chickens,...,BeefCattle Present,Chickens Present,Turkeys Present,Horses Present,Sheep/Goats Present,Animals Present,colMthTxt,refPntTxt,ColDate,geometry
0,Guthrie Center Egg Farm - Laop,2143 215TH ROAD,Guthrie Center,IA,50115-8542,Confinement,NaN,NaN,NaN,13893.0,...,False,True,False,False,False,Chickens,INTERPOLATION-PHOTO,PLANT ENTRANCE (GENERAL),2011-07-27,POINT (-94.52204999999999 41.69573)
1,"Sunrise Farms, Inc",2060 WHITE AVE,Harris,IA,51345,Confinement,NaN,NaN,NaN,80000.0,...,False,True,False,False,False,Chickens,INTERPOLATION-PHOTO,PLANT ENTRANCE (GENERAL),2017-05-18,POINT (-95.44726 43.3487)
2,Daybreak Foods Vincent Complex,3159 BUCHANAN AVE,Eagle Grove,IA,50533,Confinement,NaN,NaN,NaN,60383.0,...,False,True,False,False,False,Chickens,INTERPOLATION-PHOTO,PLANT ENTRANCE (GENERAL),2005-09-06,POINT (-93.94223 42.59563)
3,"Cedar Lane Estates, Incorporated",1114 J AVE,Jefferson,IA,50129,Confinement,1992.0,NaN,NaN,NaN,...,False,False,False,False,False,Swine,INTERPOLATION-PHOTO,PLANT ENTRANCE (GENERAL),2017-01-04,POINT (-94.46129000000001 42.06094)
4,Jbh Groeneweg Site 1,2864 Fig Ave,Rock Valley,IA,51247,Open Feedlot,NaN,NaN,700.0,NaN,...,True,False,False,False,False,BeefCattle,INTERPOLATION-PHOTO,CENTER OF FACILITY,2017-06-15,POINT (-96.27392 43.23656)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9978,Weis Family Feeders,1479 Quauk Avenue,Afton,IA,50830,Confinement,980.0,NaN,NaN,NaN,...,False,False,False,False,False,Swine,INTERPOLATION-PHOTO,CENTER OF FACILITY,2020-07-16,POINT (-94.16158 41.08844)
9979,Thompson Pork - Jake & Danielle Thompson,250th Street,Bridgewater,IA,508370000,Confinement,980.0,NaN,NaN,NaN,...,False,False,False,False,False,Swine,INTERPOLATION-PHOTO,CENTER OF FACILITY,2020-07-16,POINT (-94.63784 41.29117)
9980,Brush Creek Henhouse Llc,175th Street,Leon,IA,50144,Confinement,NaN,NaN,NaN,200.0,...,False,True,False,False,False,Chickens,INTERPOLATION-PHOTO,CENTER OF FACILITY,2020-07-30,POINT (-93.73882 40.79384)
9981,Kaufman Farms Llc,Cedar Road,Panama,IA,51562,Confinement,980.0,NaN,NaN,NaN,...,False,False,False,False,False,Swine,INTERPOLATION-PHOTO,CENTER OF FACILITY,2020-07-29,POINT (-95.31149000000001 41.69327)


#### Number of sites with each reference point type

In [3]:
active_sites_df['refPntTxt'].value_counts()

CENTER OF FACILITY                       3591
PLANT ENTRANCE (GENERAL)                  330
STORAGE TANK                                6
ADMINISTRATIVE BUILDING                     2
COMBINED ANIMAL FEED OPERATION (CAFO)       1
Name: refPntTxt, dtype: int64

#### Number of sites with no reference point type listed

In [4]:
sum(active_sites_df['refPntTxt'].isnull())

6053

## NAIP Imagery

### Timeframe coverage

#### NAIP 1 meter collection timeframe

In [5]:
naip_1m_start = '2012-01-01'
naip_1m_end = '2018-01-01'

#### Identify which sites were labeled prior to the end of the NAIP 1 meter collection

In [6]:
naip_1m_idx = []
for i in range(len(active_sites_df)):
    if active_sites_df.iloc[i].ColDate <= naip_1m_end:
        naip_1m_idx.append(i)

In [7]:
len(naip_1m_idx)

9491

#### NAIP 0.6 meter collection timeframe

In [8]:
naip_06m_start = '2018-01-01'
naip_06m_end = '2018-12-31'

#### Identify which sites were labeled prior to the end of the NAIP 0.6 meter collection

In [9]:
naip_06m_idx = []
for i in range(len(active_sites_df)):
    if active_sites_df.iloc[i].ColDate <= naip_06m_end:
        naip_06m_idx.append(i)

In [10]:
len(naip_06m_idx)

9820

### Identify the scenes completely covering a 1 kilometer bounding box centered on the site coordinates

#### Helper function defining the geojson specification for the bounding box

In [11]:
def bbox_coordinates(long,lat,bbox_length):
    """ Returns a geojson specification of a bounding box centered on (lat,long) of size bbox_length meters """
    
    # Conversion factor in degrees per meter
    dpm = 9e-6
    
    # Compute the delta in latitude
    delta_lat = dpm*bbox_length/2
    
    # Compute the delta in longitude
    delta_long = dpm*bbox_length/(2*cos(lat))
    
    # Prepare the geojson specification
    spec = { "type": "Feature",
             "bbox": [long-delta_long, lat-delta_lat, long+delta_long, lat+delta_lat],
             "geometry": {
              "type": "Polygon",
              "coordinates": [[
                [long-delta_long, lat-delta_lat], [long-delta_long, lat+delta_lat], 
                [long+delta_long, lat+delta_lat], [long+delta_long, lat-delta_lat]
                ]]
              }
            }
    
    return spec

#### NAIP 1 meter collection

In [12]:
naip_1m_site_scenes = defaultdict(list)
j = 0
for i in naip_1m_idx:
    
    # Get the site record
    site = active_sites_df.iloc[i]
    
    # Get the site coordinates
    (site_long,site_lat) = site.geometry.coords[0]

    # Compute the coordinates of the 1 kilometer bounding box
    site_bbox = bbox_coordinates(site_long,site_lat,1e3)

    # Search for imagery covering the site bounding box
    scenes, ctx = dl.scenes.search(
        aoi=site_bbox,
        products=["usda:naip:rgbn:v1"],
        start_datetime=naip_1m_start,
        end_datetime=naip_1m_end
    )
    
    # Save the scenes where the bounding box is completely covered
    for scene in scenes:
        if scene.coverage(site_bbox) == 1.0:
            naip_1m_site_scenes[i].append(scene)
    
    j += 1
    if j % 1000 == 0:
        print(f"Processed {j} sites.")

Processed 1000 sites.
Processed 2000 sites.
Processed 3000 sites.
Processed 4000 sites.
Processed 5000 sites.
Processed 6000 sites.
Processed 7000 sites.
Processed 8000 sites.
Processed 9000 sites.


#### NAIP 0.6 meter collection

In [13]:
"""
# Pick just one scene
scene = scenes[0]

# Load the data as an ndarray
arr = scene.ndarray("red green blue", ctx)

# Display the scene
dl.scenes.display(arr, title=scene.properties.id)
"""

'\n# Pick just one scene\nscene = scenes[0]\n\n# Load the data as an ndarray\narr = scene.ndarray("red green blue", ctx)\n\n# Display the scene\ndl.scenes.display(arr, title=scene.properties.id)\n'